In [2]:
import pydicom
import numpy as np
import os
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


In [6]:
def load_dicom_series(dicom_dir : str) -> np.ndarray: 
    # list files
    dicom_files = sorted([os.path.join(dicom_dir, f) for f in os.listdir(dicom_dir) if f.endswith('.DCM')])
    # read dcm 
    slices = [pydicom.dcmread(dcm) for dcm in dicom_files]
    slices.sort(key=lambda x: int(x.InstanceNumber))  # Sort by slice order
    # stack and make into 3d volume
    volume = np.stack([s.pixel_array for s in slices], axis=-1)
    return volume

# Load DICOM data
DICOM_DIR = "/Users/akshdeepsandhu/Desktop/MRI/imrh_114a/imoco_recon_0"  
volume = load_dicom_series(DICOM_DIR)
dimensions = volume.shape

In [ ]:

# Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("DICOM Viewer"),
    
    html.Div([
        html.Label("Select Axis:"),
        dcc.Dropdown(
            id='axis-selector',
            options=[
                {'label': 'X-Axis', 'value': 'x'},
                {'label': 'Y-Axis', 'value': 'y'},
                {'label': 'Z-Axis', 'value': 'z'}
            ],
            value='z'
        )
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Slice Index:"),
        dcc.Slider(
            id='slice-slider',
            min=0,
            max=dimensions[2] - 1,
            step=1,
            value=0,
            marks={i: str(i) for i in range(0, dimensions[2], max(1, dimensions[2] // 10))}
        )
    ], style={'width': '60%', 'display': 'inline-block', 'padding': '0px 20px'}),
    
    html.Div([
        html.Label("Adjust Contrast:"),
        dcc.Slider(
            id='contrast-slider',
            min=0,
            max=1.5,
            step=0.01,
            value=1.0,
            marks={i: f"{i}" for i in range(0, 1)}
        )
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    dcc.Graph(id='slice-display', style={'height': '600px'})  
])

@app.callback(
    Output('slice-display', 'figure'),
    [Input('axis-selector', 'value'),
     Input('slice-slider', 'value'),
     Input('contrast-slider', 'value')]
)
def update_slice(axis, index, contrast):
    if axis == 'x':
        slice_2d = volume[index, :, :]
        title = f'Slice at X={index}'
    elif axis == 'y':
        slice_2d = volume[:, index, :]
        title = f'Slice at Y={index}'
    elif axis == 'z':
        slice_2d = volume[:, :, index]
        title = f'Slice at Z={index}'
    else:
        slice_2d = np.zeros_like(volume[:, :, 0])  
        title = "Invalid Axis"

    # contrast
    slice_2d = np.clip(slice_2d * contrast, 0, 255)  
    # rotate
    slice_2d = np.rot90(slice_2d,k=3)

    # plot
    fig = px.imshow(slice_2d, color_continuous_scale='gray')
    fig.update_layout(
        title=title,
        coloraxis_showscale=False,
        margin=dict(l=10, r=10, t=40, b=10)  
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
